### Scrapes block reward values

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [4]:
URL = 'https://bitinfocharts.com/comparison/size-btc.html#alltime'
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')
scripts = soup.find_all('script')

In [7]:
def parse(string_list):
    '''
    parse list of strings within the script tag
    [date, volume]
    '''
    clean = re.sub('[\[\],\s]', '', string_list)
    splitted = re.split("[\'\"]", clean)
    values_only = [s for s in splitted if s != '']
    return values_only

def process_scripts():
    '''
    Scrape URL script tag and extract block reward & respective date
    '''
    dates = []
    sizes = []
    
    for script in scripts:
        if 'd = new Dygraph(document.getElementById("container")' in script.text:
            str_lst = script.text
            str_lst = '[[' + str_lst.split('[[')[-1]
            str_lst = str_lst.split(']]')[0] +']]'
            str_lst = str_lst.replace('new Date(', '').replace(')', '')
            data = parse(str_lst)
    
    for each in data:
        if (data.index(each) % 2) == 0:
            dates.append(each)
        else:
            sizes.append(each)
            
    return dates, sizes

def create_dataframe():
    '''
    Create dataframe from scraped block reward sizes and dates
    '''
    dates, sizes = process_scripts()
    df = pd.DataFrame(list(zip(dates, sizes)), columns=['Date', 'Block Reward Size'])
    return df

def export_data(df):
    '''
    Save data
    '''
    df.to_csv('../../data/BTC_Block_Reward.csv')

In [8]:
df = create_dataframe()

In [10]:
df.tail()

,Date,Block Reward Size
5097,2022/12/18,511467
5098,2022/12/19,706111
5099,2022/12/20,665567
5100,2022/12/21,653348
5101,2022/12/22,676142


In [11]:
export_data(df)